In [273]:
import pandas as pd

In [274]:
df_1 = pd.read_csv('MANUSCRITO_1-UTF8.csv', sep=';')
print(df_1.shape)
df_1

(10000, 53)


,idBNE,Autor Personas,Autor Entidades,Título uniforme,Título,Otros Títulos,Información asociada al título,Edición,Editorial,Editor musical,...,Depósito Legal,Otros Números Normalizados,Número Bibliografía Nacional,País de publicación,Lengua de publicación,Lengua original,otras lenguas,Tipo de documento,Condiciones de uso,enlaces
0,biam0000000003,"Isidoro, Santo, Arzobispo de Sevilla //",NaN,[Sententiarum libri tres. Español] //,Del soberano bien,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,lat,spa //,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000054150&p...
1,biam0000000005,NaN,NaN,NaN,Tratado de minas [militares],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000054147&p...
2,biam0000000007,"Figueroa, Francisco de (S.I.) //",NaN,NaN,"Vida del Venerable P. Juan Sebastián, religios...",NaN,"escrita por el P. Francisco de Figueroa, de la...",NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000054146&p...
3,biam0000000008,Josefa Victoria (O.F.M. Cap.) //,NaN,NaN,Fundación del Monasterio de Capuchinas de Jesú...,Fundación del Monasterio de Capuchinas de Jesú...,"y ordenó y dispuso, añadiendo algunas cosas po...",NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,spa //,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000139711&p...
4,biam0000000011,"Laudivius, Zacharias //",NaN,NaN,Liber epistolarum Magni Turci [Mahomet II],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,lat,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000054068&p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,biam0000013688,NaN,NaN,[Biblia.. A.T.. Salmos. Latín] //,Psalterium integrum Libris V,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,lat //,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000014895&p...
9996,biam0000013689,NaN,Monasterio de las Descalzas Reales (Madrid) //,[Consueta de las Descalzas Reales de Madrid] //,Regla de lo que se debe hacer y cantar en la C...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000238007&p...
9997,biam0000013690,NaN,NaN,NaN,Amors de Curial e Güelfa. Inc.: Quant es gran ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,cat,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000068666&p...
9998,biam0000013691,"Valera, Juan , 1824-1905 //",NaN,NaN,Cartas de Juan Valera a Francisco A. Barbieri,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000181023&p...


In [275]:
df_1.shape

(10000, 53)

In [276]:
df_1 = df_1.dropna(how='all', axis=1)

In [277]:
df_1 = df_1[df_1['Autor Personas'].notna()]

In [278]:
df_1.shape

(6110, 35)

In [279]:
df_1['Título'].isna().sum()

0

In [280]:
df_1.isna().sum().sort_values(ascending=False)[:25]

Premios                                          6109
Depósito Legal                                   6109
Editorial                                        6108
Número Bibliografía Nacional                     6106
información asociada a registros relacionados    6101
id registros relacionados                        6101
Condiciones de uso                               6100
id otras ediciones                               6094
información asociada a otras ediciones           6094
Lugar relacionado                                6090
Resumen                                          6081
Lengua original                                  6062
Lugar de publicación                             5995
CDU                                              5964
Otros Títulos                                    5919
Autor Entidades                                  5830
otras lenguas                                    5788
transcripciones de interés                       5489
Título uniforme             

In [281]:
import re

def clean(s):
    s = re.sub('\d','',s)
    s = re.sub('\(.*\)', '', s)
    s = re.sub(',', '_', s) # tengo que cambiar las comas porque luego
    return s

In [282]:
df_1['Autor Personas'] = df_1['Autor Personas'].apply(lambda x: clean(x))

In [283]:
df_1['Autor Personas'] = df_1['Autor Personas'].str.lower()

In [284]:
df_1['Autor Personas']

0                isidoro_ santo_ arzobispo de sevilla  //
2                             figueroa_ francisco de   //
3                                    josefa victoria   //
4                                laudivius_ zacharias  //
5                            paula de jesús nazareno   //
                              ...                        
9989                          juan i_ rey de castilla  //
9991                      tramoyeres blasco_ luis _ -  //
9992                         rojas_ josé maría de _ -  //
9994    silvela_ francisco _ -  // toreno_ josé maría ...
9998                                 valera_ juan _ -  //
Name: Autor Personas, Length: 6110, dtype: object

In [285]:
# la ultima posicion del string siempre va a ser vacio
df_1['Autor Personas'] = df_1['Autor Personas'].str.split('//').apply(lambda x: x[:-1])

In [286]:
df_1 = df_1.explode('Autor Personas') # si hay varios autores duplico la fila 

In [287]:
def aux(l):
    # la ultima posicion suele ser el titulo de la persona
    if len(l) == 1:
        return l
    else:
        surname = l[0] # primero va el nombre y despues el apellido
        l[0] = l[1]
        l[1] = surname
        return l

In [288]:
# reordenar nombre y apellido
df_1['Autor Personas'] = df_1['Autor Personas'].str.split('_').apply(lambda x: aux(x))

In [289]:
df_1['Autor Personas']


0              [ santo, isidoro,  arzobispo de sevilla  ]
2                            [ francisco de   , figueroa]
3                                    [josefa victoria   ]
4                               [ zacharias  , laudivius]
5                            [paula de jesús nazareno   ]
                              ...                        
9991                    [ luis , tramoyeres blasco,  -  ]
9992                       [ josé maría de , rojas,  -  ]
9994                         [ francisco , silvela,  -  ]
9994    [ josé maría queipo de llano ruiz de saravia, ...
9998                               [ juan , valera,  -  ]
Name: Autor Personas, Length: 8172, dtype: object

In [290]:
def aux_2(l):
    for i in range(len(l)):
        l[i] = re.sub('^\ +', '', l[i]) # quitar espacios al principio
        l[i] = re.sub('\ +$', '', l[i]) # quitar espacios al final
        l[i] = re.sub(' ', '_', l[i])
        # l[i] = re.sub('^(?!\w)', '', l[i])
    return l

In [291]:
# quitar espacios en blanco dentro de cada posicion a no ser que sea entre palabras
df_1['Autor Personas'] = df_1['Autor Personas'].apply(lambda x: aux_2(x))
df_1['Autor Personas']

0                  [santo, isidoro, arzobispo_de_sevilla]
2                                [francisco_de, figueroa]
3                                       [josefa_victoria]
4                                  [zacharias, laudivius]
5                               [paula_de_jesús_nazareno]
                              ...                        
9991                         [luis, tramoyeres_blasco, -]
9992                            [josé_maría_de, rojas, -]
9994                              [francisco, silvela, -]
9994    [josé_maría_queipo_de_llano_ruiz_de_saravia, t...
9998                                    [juan, valera, -]
Name: Autor Personas, Length: 8172, dtype: object

In [292]:
df_1['Autor Personas'] = df_1['Autor Personas'].apply(lambda x: "_". join(x))
df_1['Autor Personas']

0                      santo_isidoro_arzobispo_de_sevilla
2                                   francisco_de_figueroa
3                                         josefa_victoria
4                                     zacharias_laudivius
5                                 paula_de_jesús_nazareno
                              ...                        
9991                             luis_tramoyeres_blasco_-
9992                                josé_maría_de_rojas_-
9994                                  francisco_silvela_-
9994    josé_maría_queipo_de_llano_ruiz_de_saravia_tor...
9998                                        juan_valera_-
Name: Autor Personas, Length: 8172, dtype: object

In [298]:
import unicodedata

def strip_accents(s):
    # https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [300]:
df_1['Autor Personas'] = df_1['Autor Personas'].apply(lambda x: strip_accents(x))
df_1['Autor Personas']

0                      santo_isidoro_arzobispo_de_sevilla
2                                   francisco_de_figueroa
3                                         josefa_victoria
4                                     zacharias_laudivius
5                                 paula_de_jesus_nazareno
                              ...                        
9991                             luis_tramoyeres_blasco_-
9992                                jose_maria_de_rojas_-
9994                                  francisco_silvela_-
9994    jose_maria_queipo_de_llano_ruiz_de_saravia_tor...
9998                                        juan_valera_-
Name: Autor Personas, Length: 8172, dtype: object

In [303]:
df_1['Autor Personas'] = df_1['Autor Personas'].apply(lambda x: re.sub('[^a-zA-Z_]', '', x))
df_1['Autor Personas']

0                      santo_isidoro_arzobispo_de_sevilla
2                                   francisco_de_figueroa
3                                         josefa_victoria
4                                     zacharias_laudivius
5                                 paula_de_jesus_nazareno
                              ...                        
9991                              luis_tramoyeres_blasco_
9992                                 jose_maria_de_rojas_
9994                                   francisco_silvela_
9994    jose_maria_queipo_de_llano_ruiz_de_saravia_tor...
9998                                         juan_valera_
Name: Autor Personas, Length: 8172, dtype: object

In [304]:
df_1['Autor Personas'] = df_1['Autor Personas'].apply(lambda x: re.sub('_$', '', x))
df_1['Autor Personas']

0                      santo_isidoro_arzobispo_de_sevilla
2                                   francisco_de_figueroa
3                                         josefa_victoria
4                                     zacharias_laudivius
5                                 paula_de_jesus_nazareno
                              ...                        
9991                               luis_tramoyeres_blasco
9992                                  jose_maria_de_rojas
9994                                    francisco_silvela
9994    jose_maria_queipo_de_llano_ruiz_de_saravia_tor...
9998                                          juan_valera
Name: Autor Personas, Length: 8172, dtype: object

In [ ]:
# CLEAN NAAAAAAAAAAMES

In [1]:
import pandas as pd
import re
import unicodedata

AUTHOR_COL = 'Autor Personas'

def handler(event, context):
    #TODO extract info from context
    csv = '.csv'
    df = pd.read_csv('MANUSCRITO_1-UTF8.csv', sep=';')
    df = drop_rows_and_cols(df)
    df = clean_author_names(df)
    return df
    
def drop_rows_and_cols(df):
    """Drop empty rows and columns. Also drop rows without an author"""
    df = df.copy()
    df = df.dropna(how='all', axis=0)
    df = df.dropna(how='all', axis=1)
    df = df[df[AUTHOR_COL].notna()]
    return df

def clean_author_names(df):
    df = df.copy()
    
    
    df[AUTHOR_COL] = (
        df[AUTHOR_COL]
        .apply(lambda x: clean(x))
        .str.split('//')
        .apply(lambda x: remove_empty_list_position(x))
        .explode(AUTHOR_COL)
        .str.split('_')
        .apply(lambda x: swap_name_and_surname(x))
        .apply(lambda x: treat_whitespaces(x))
        .apply(lambda x: "_". join(x))
        .apply(lambda x: strip_accents(x))
        .apply(lambda x: keep_chars_and_separator(x))
        .apply(lambda x: clean_ending(x))
    )
    
    
    
    # df[AUTHOR_COL] = df[AUTHOR_COL].apply(lambda x: clean(x))
    # # df_1[AUTHOR_COL] = df_1[AUTHOR_COL].str.lower()
    
    # # all authors end with "//" if there is more than one author it is written after "//": 
    # #   Manolo // Pepe//
    # # take all positions of the array except for the last one cause it is empty: 
    # #   Manolo //.split("//") -> [Manolo, ]
    # df[AUTHOR_COL] = df[AUTHOR_COL].str.split('//').apply(lambda x: x[:-1])
    # df = df.explode(AUTHOR_COL) # if more than one author duplicate the row 
    # df[AUTHOR_COL] = df[AUTHOR_COL].str.split('_').apply(lambda x: swap_name_and_surname(x))
    # df[AUTHOR_COL] = df[AUTHOR_COL].apply(lambda x: treat_whitespaces(x))
    # df[AUTHOR_COL] = df[AUTHOR_COL].apply(lambda x: "_". join(x)) # [paco, de_lucia] -> paco_de_lucia
    # df[AUTHOR_COL] = df[AUTHOR_COL].apply(lambda x: strip_accents(x))
    # df[AUTHOR_COL] = df[AUTHOR_COL].apply(lambda x: re.sub('[^a-zA-Z_]', '', x)) # ensure we only have alphanumeric characters
    # df[AUTHOR_COL] = df[AUTHOR_COL].apply(lambda x: re.sub('_$', '', x)) # at this point there could be remaining underscores at the end
    return df

def clean(s):
    s = re.sub('\d','',s) # get rid of digits
    s = re.sub('\(.*\)', '', s) # get rid of parenthesis and what is inside
    s = re.sub(',', '_', s) # need to change comas for other symbol cause they are problematic when splitting an string
    s = s.lower()
    return s


def remove_empty_list_position(l):
    return l[:1]


def swap_name_and_surname(l):
    # the format is [surname, name, title]
    if len(l) == 1:
        return l
    else:
        surname = l[0] 
        l[0] = l[1]
        l[1] = surname
        return l

def treat_whitespaces(l):
    for i in range(len(l)):
        l[i] = re.sub('^\ +', '', l[i])
        l[i] = re.sub('\ +$', '', l[i])
        l[i] = re.sub(' ', '_', l[i]) # substitute the remaining whitespaces with _: de lucia -> de_lucia
        # l[i] = re.sub('^(?!\w)', '', l[i])
    return l


def strip_accents(s):
    # https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def keep_chars_and_separator(s):
    return re.sub('[^a-zA-Z_]', '', s)

def clean_ending(s):
    return re.sub('_$', '', s)

In [5]:
handler('','').isna().sum()

idBNE                                               0
Autor Personas                                   2609
Autor Entidades                                  5830
Título uniforme                                  5410
Título                                              0
Otros Títulos                                    5919
Información asociada al título                   4031
Editorial                                        6108
Lugar de publicación                             5995
Fecha de publicación                                6
Descripción física                                  0
Contenido                                        5270
Descripción y notas                               567
Citas o referencias                              3886
transcripciones de interés                       5489
Procedencia                                      2677
Resumen                                          6081
Premios                                          6109
Tema                        

In [6]:
df = handler('','')
df

,idBNE,Autor Personas,Autor Entidades,Título uniforme,Título,Otros Títulos,Información asociada al título,Editorial,Lugar de publicación,Fecha de publicación,...,información asociada a otras ediciones,Depósito Legal,Número Bibliografía Nacional,País de publicación,Lengua de publicación,Lengua original,otras lenguas,Tipo de documento,Condiciones de uso,enlaces
0,biam0000000003,santo_isidoro_arzobispo_de_sevilla,NaN,[Sententiarum libri tres. Español] //,Del soberano bien,NaN,NaN,NaN,NaN,entre 1400 y 1499?,...,NaN,NaN,NaN,sp,spa,lat,spa //,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000054150&p...
2,biam0000000007,josefa_victoria,NaN,NaN,"Vida del Venerable P. Juan Sebastián, religios...",NaN,"escrita por el P. Francisco de Figueroa, de la...",NaN,NaN,entre 1700 y 1799?,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000054146&p...
3,biam0000000008,zacharias_laudivius,NaN,NaN,Fundación del Monasterio de Capuchinas de Jesú...,Fundación del Monasterio de Capuchinas de Jesú...,"y ordenó y dispuso, añadiendo algunas cosas po...",NaN,NaN,1722,...,NaN,NaN,NaN,sp,spa,NaN,spa //,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000139711&p...
4,biam0000000011,paula_de_jesus_nazareno,NaN,NaN,Liber epistolarum Magni Turci [Mahomet II],NaN,NaN,NaN,NaN,entre 1400 y 1499?,...,NaN,NaN,NaN,sp,lat,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000054068&p...
5,biam0000000013,hieronimus_olzignanus,NaN,NaN,Epítome historial de la vida espiritual que si...,Vida de la M. Paula de Jesús Nazareno //,NaN,NaN,NaN,entre 1700 y 1799?,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000054078&p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,biam0000013680,NaN,Castilla (Reino) Cortes //,NaN,Traslado de las leyes pertenecientes a los pre...,"Cortes de Guadalajara, 1392",NaN,NaN,NaN,1753?,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,NaN
9991,biam0000013683,NaN,NaN,NaN,Cartas de Luís Tramoyeres a Francisco A. Barbieri,NaN,NaN,NaN,NaN,1882,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000258990&p...
9992,biam0000013684,NaN,NaN,NaN,Carta y tarjetas de visita de José María Rojas...,NaN,Rojas,NaN,NaN,1874,...,NaN,NaN,NaN,NaN,spa,NaN,NaN,Manuscrito,NaN,NaN
9994,biam0000013687,NaN,NaN,NaN,Papeles referentes al reglamento de policia te...,NaN,NaN,NaN,NaN,1879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manuscrito,NaN,NaN


In [293]:
df_1.columns

Index(['idBNE', 'Autor Personas', 'Autor Entidades', 'Título uniforme',
       'Título', 'Otros Títulos', 'Información asociada al título',
       'Editorial', 'Lugar de publicación', 'Fecha de publicación',
       'Descripción física', 'Contenido', 'Descripción y notas',
       'Citas o referencias', 'transcripciones de interés', 'Procedencia',
       'Resumen', 'Premios', 'Tema', 'Género/Forma', 'Lugar relacionado',
       'CDU', 'id registros relacionados',
       'información asociada a registros relacionados', 'id otras ediciones',
       'información asociada a otras ediciones', 'Depósito Legal',
       'Número Bibliografía Nacional', 'País de publicación',
       'Lengua de publicación', 'Lengua original', 'otras lenguas',
       'Tipo de documento', 'Condiciones de uso', 'enlaces'],
      dtype='object')

In [12]:
df[df['Título uniforme'].notna()]['Título uniforme']

0                   [Sententiarum libri tres. Español] //
8       [Actiones quas pro Hispana corona Gallae antep...
9                                   [De las librerías] //
10      [Vida del P. Pedro Urraca de la Santísima Trin...
11      [Historia del Duende de Palacio] // [Vida del ...
                              ...                        
9955                                           [Obras] //
9960                                      [Passionale] //
9972    [Theatro crítico universal o Discursos varios ...
9973                    [Biblia.. A.T.. Salmos. Latín] //
9988                                         [Digesta] //
Name: Título uniforme, Length: 700, dtype: object

In [38]:
df[df['Tipo de documento'].notna()]['Tipo de documento']

0       Manuscrito
2       Manuscrito
3       Manuscrito
4       Manuscrito
5       Manuscrito
           ...    
9989    Manuscrito
9991    Manuscrito
9992    Manuscrito
9994    Manuscrito
9998    Manuscrito
Name: Tipo de documento, Length: 6110, dtype: object

In [44]:
df.iloc[0,:].to_json('./try.json')

In [47]:

book = df.iloc[0,:].to_dict()
book 

{'idBNE': 'biam0000000003',
 'Autor Personas': 'santo_isidoro_arzobispo_de_sevilla',
 'Autor Entidades': nan,
 'Título uniforme': '[Sententiarum libri tres. Español] //',
 'Título': 'Del soberano bien',
 'Otros Títulos': nan,
 'Información asociada al título': nan,
 'Editorial': nan,
 'Lugar de publicación': nan,
 'Fecha de publicación': 'entre 1400 y 1499?',
 'Descripción física': '5, 195 h. , 22 x 15 cm ,',
 'Contenido': nan,
 'Descripción y notas': 'Foliación errónea: de la h. 171 pasa a la 173, y de la 188 a la 190.  // Iniciales en azul y rojo. Títulos de capítulos en rojo  //',
 'Citas o referencias': 'Domínguez Bordona, Mss. con pinturas, I p. 432, nº 1005  // Alvar y Lucía, Literatura medieval p. 905-906  // García Rives, Luis, Estudio de las traducciones castellanas de obras de San Isidoro, en RABM (1950) p. 299  // Grespi, Giuseppina, Traducciones castellanas de obras latinas e italianas contenidas en manuscritos del siglo XV en las bibliotecas de Madrid y El Escorial. Madrid

In [45]:
for idx, row in df.iterrows():
    print(idx)
    print(row)
    break

0
idBNE                                                                               biam0000000003
Autor Personas                                                  santo_isidoro_arzobispo_de_sevilla
Autor Entidades                                                                                NaN
Título uniforme                                              [Sententiarum libri tres. Español] //
Título                                                                           Del soberano bien
Otros Títulos                                                                                  NaN
Información asociada al título                                                                 NaN
Editorial                                                                                      NaN
Lugar de publicación                                                                           NaN
Fecha de publicación                                                            entre 1400 y 1499?
Descripc

In [ ]:
'idBNE', 'Autor Personas', 'Autor Entidades',
       'Título',
       'Editorial', 'Lugar de publicación', 'Fecha de publicación',
       'Citas o referencias',
       'Premios', 'Tema', 'Género/Forma', 'Lugar relacionado',
       'id registros relacionados',
       
        
       'Número Bibliografía Nacional', 'País de publicación',
       'Lengua de publicación', 'Lengua original', 'otras lenguas',
       'Tipo de documento',

In [294]:
df_1.dtypes

idBNE                                            object
Autor Personas                                   object
Autor Entidades                                  object
Título uniforme                                  object
Título                                           object
Otros Títulos                                    object
Información asociada al título                   object
Editorial                                        object
Lugar de publicación                             object
Fecha de publicación                             object
Descripción física                               object
Contenido                                        object
Descripción y notas                              object
Citas o referencias                              object
transcripciones de interés                       object
Procedencia                                      object
Resumen                                          object
Premios                                         

In [295]:
df_2 = pd.read_csv('MANUSCRITO_2-UTF8.csv', sep=';')
print(df_2.shape)
df_2

(10000, 53)


,idBNE,Autor Personas,Autor Entidades,Título uniforme,Título,Otros Títulos,Información asociada al título,Edición,Editorial,Editor musical,...,Depósito Legal,Otros Números Normalizados,Número Bibliografía Nacional,País de publicación,Lengua de publicación,Lengua original,otras lenguas,Tipo de documento,Condiciones de uso,enlaces
0,biam0000013693,NaN,NaN,NaN,Registro de títulos de personal de la Sección ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000238305&p...
1,biam0000013694,"Sabatini, Francisco , 1722-1797 // Chone de A...",NaN,NaN,Oficios e informaciones de Francisco Sabatini ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,spa,NaN,NaN,Manuscrito,NaN,NaN
2,biam0000013695,"Yepes, Francisco de (O.F.M.) , fl. 1558 //",NaN,NaN,Tratados varios,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000238152&p...
3,biam0000013697,"Zubiaurre, Valentín de , 1837-1914 //",NaN,NaN,Cartas y tarjeta de visita de Valentín María d...,NaN,V. Zubiaurre,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,spa,NaN,NaN,Manuscrito,NaN,NaN
4,biam0000013698,"Zozaya, Benito , 1844-1904 //",NaN,NaN,Cartas y tarjetas de visita de Benito Zozaya a...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,spa,NaN,NaN,Manuscrito,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,a4551440,"Villanova, Jesús // Herrero, Mariano (Herrero...",NaN,NaN,¿Quiere usted asistir al Salón Variedades? apr...,NaN,letra [de] Jesús Villanova y M. Herrero; músic...,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000221415&p...
9996,a4551442,"Prieto, Enrique , m. 1914 // Caba, Manuel , m...",NaN,NaN,"El payaso zarzuela cómica en un acto, original...",NaN,"Enrique Prieto, Manuel Caba; [música de] Ramón...",NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000210295&p...
9997,a4551451,NaN,NaN,NaN,El portal de casa grande sainete. Emp.: Que se...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000222872&p...
9998,a4551457,"Calle Guibert, Carlos de la //",NaN,NaN,Paco el contrabandista comedia en dos actos y ...,NaN,por Carlos de la Calle i Guibert,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000221034&p...


In [296]:
df_2.isna().sum().sort_values(ascending=False)[:25]

Forma de la composición sonora             10000
Otros Números Normalizados                 10000
información asociada a título anterior     10000
id título anterior                         10000
Otra clasificación                         10000
Escala del mapa                            10000
Proyección del mapa                        10000
Forma de la composición notada             10000
ISSN                                       10000
Periodicidad                               10000
información asociada a título posterior    10000
Intérpretes                                10000
id título posterior                        10000
Editor musical                             10000
Serie                                       9999
Depósito Legal                              9999
tipo material cartográfico                  9999
ISBN                                        9999
Edición                                     9997
Premios                                     9997
Lugar relacionado   

In [297]:
df_3 = pd.read_csv('MANUSCRITO_3-UTF8.csv', sep=';')
print(df_3.shape)
df_3

(10000, 53)


,idBNE,Autor Personas,Autor Entidades,Título uniforme,Título,Otros Títulos,Información asociada al título,Edición,Editorial,Editor musical,...,Depósito Legal,Otros Números Normalizados,Número Bibliografía Nacional,País de publicación,Lengua de publicación,Lengua original,otras lenguas,Tipo de documento,Condiciones de uso,enlaces
0,a4551469,NaN,NaN,NaN,El pleito del gallego sainete nuevo. Emp.: Ami...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000222543&p...
1,a4551790,"Vallejo, José Mariano , m. 1911 //",NaN,NaN,El artículo 33 discusión bufo-político-monárqu...,El artículo treinta y tres //,original de José Mariano Vallejo y Compañía,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000223034&p...
2,a4551876,"Velázquez y Sánchez, José , 1826-1880 //",NaN,NaN,Argentina poema dramático. Emp.: De la multitu...,NaN,por José Velázquez y Sánchez,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000223015&p...
3,a4551928,"Catalina, Juan , 1830-1873? //",NaN,NaN,Así son todas apropósito en un acto. Emp.: Ver...,NaN,arreglado del francés por Juan Catalina,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000223041&p...
4,a4551958,"Jackson Veyán, José , 1852-1935 //",NaN,NaN,La piedra filosofal juguete cómico en un acto ...,NaN,[José Jackson Veyán],NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,a5001534,NaN,NaN,NaN,España felicitada con dos remedios solos tan p...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000015844&p...
9996,a5001540,"Tarafa, Francisco //",NaN,NaN,Crònica de la provincia de Catalunya en la cit...,Crónica de Cataluña //,"compilada per Francesch Tarapha, canonge de Ba...",NaN,NaN,NaN,...,NaN,NaN,NaN,sp,cat,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000015426&p...
9997,a5001573,NaN,NaN,NaN,Papeles referentes al pleito de los Capuchinos...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000011523&p...
9998,a5001574,"García de Ercilla, Fortún //",NaN,NaN,Discurso histórico y jurídico del desafío del ...,NaN,compuesto de orden del mismo señor Emperador p...,NaN,NaN,NaN,...,NaN,NaN,NaN,sp,spa,NaN,NaN,Manuscrito,NaN,http://bdh-rd.bne.es/viewer.vm?id=0000109434&p...
